In [72]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import gym
from collections import deque
import random
import time
import yaml
from klqr import *
from train_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
with open('config.yml','r') as ymlfile:
    config = yaml.load(ymlfile)
    
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [76]:
env = gym.make('Pendulum-v0')
agent = klqr(config,sess)
agent.build_model()
train_agent(agent,env,max_timesteps=40000, n_transitions_between_updates=100,
                n_optim_steps_per_update=200,
                n_iters_per_p_update=5,
                )

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
z shape: (?, 8)
********** Iteration 1 ************
	Episode Len: 0
	Last Episode Cost: 0.000000 vs optimal 0.000000
********** Iteration 2 ************
	Episode Len: 200
	Last Episode Cost: 979.921306 vs optimal 0.000000
********** Iteration 3 ************
	Episode Len: 200
	Last Episode Cost: 979.921306 vs optimal 0.000000
********** Iteration 4 ************
	Episode Len: 200
	Last Episode Cost: 1047.876621 vs optimal 0.000000
********** Iteration 5 ************
[[ 1.1772047e+00  1.1984469e+00  3.2233259e-01 -1.9844962e-02
   8.4431094e-01 -2.0724992e-01  6.2711352e-01  1.3447186e-01]
 [ 1.1984469e+00  9.8515110e+00  2.0909231e+00 -5.3377146e-01
   5.9627681e+00 -1.0780185e+00  3.3027630e+00  4.3140608e-01]
 [ 3.2233259e-01  2.0909231e+00  1.4798940e+00 -1.1225759e-01
   1.4734429e+00 -2.6

InvalidArgumentError: Cholesky decomposition was not successful. The input might not be valid.
	 [[Node: model/Cholesky = Cholesky[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](model/P/read)]]

Caused by op 'model/Cholesky', defined at:
  File "/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.6/site-pack

In [68]:
#test

agent.max_a = 100.0
agent.min_a = -100.0
train_agent(agent,env,max_timesteps=1000,training=False)

********** Iteration 1 ************


InvalidArgumentError: Input is not invertible.
	 [[Node: model/compute_K/MatrixInverse = MatrixInverse[T=DT_FLOAT, adjoint=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](model/compute_K/add)]]

Caused by op 'model/compute_K/MatrixInverse', defined at:
  File "/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.6/

In [13]:
#test

agent.max_a = 100.0
agent.min_a = -100.0
train_agent(agent,env,max_timesteps=1000,training=False)

********** Iteration 1 ************
	Episode Len: 100
	Last Episode Cost: 15.114967 vs optimal 0.000000
********** Iteration 2 ************
	Episode Len: 100
	Last Episode Cost: 2.847836 vs optimal 2.355781
********** Iteration 3 ************
	Episode Len: 100
	Last Episode Cost: 12.386197 vs optimal 11.864259
********** Iteration 4 ************
	Episode Len: 100
	Last Episode Cost: 13.126852 vs optimal 11.716004
********** Iteration 5 ************
	Episode Len: 100
	Last Episode Cost: 8.160144 vs optimal 7.246125
********** Iteration 6 ************
	Episode Len: 100
	Last Episode Cost: 12.285117 vs optimal 11.117412
********** Iteration 7 ************
	Episode Len: 100
	Last Episode Cost: 8.954077 vs optimal 7.739204
********** Iteration 8 ************
	Episode Len: 100
	Last Episode Cost: 14.499120 vs optimal 13.201736
********** Iteration 9 ************
	Episode Len: 100
	Last Episode Cost: 5.719255 vs optimal 5.130324
********** Iteration 10 ************
	Episode Len: 100
	Last Epi

In [69]:
sess.run(agent.A)

array([[nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan]], dtype=float32)

In [70]:
sess.run(agent.B)

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]], dtype=float32)

In [71]:
sess.run(agent.Q)

array([[nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan]], dtype=float32)

In [38]:
sess.run(agent.R)

array([[0.17767526]], dtype=float32)

In [40]:
A,B,Q,R = (env.env.A, env.env.B, env.env.Q, env.env.R)
P = Q
for k in range(100):
    P = Q + A.T @ P @ A - A.T @ P.T @ B @ np.linalg.inv(R + B.T @ P @ B ) @ B.T @ P @ A
    P = 0.5*(P + P.T)

In [41]:
P

array([[2.23999771, 0.54694411, 0.19286546],
       [0.54694411, 2.35082156, 0.54694411],
       [0.19286546, 0.54694411, 2.23999771]])

In [42]:
sess.run(agent.P)

array([[1.7117685 , 0.55636847, 0.07273098],
       [0.55636847, 1.7515693 , 0.6495986 ],
       [0.07273098, 0.6495986 , 1.8076872 ]], dtype=float32)